# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком

Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.

## В этом задании вы:

 + вспомните, что такое t-критерий Стьюдента и для чего он применяется
 + сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
 + почувствуете разницу в результатах применения различных методов поправки на множественную проверку

In [12]:
import pandas as pd
import numpy as np
import scipy
import statsmodels
from statsmodels.sandbox.stats.multicomp import multipletests 

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [3]:
control_data = data[data.Diagnosis == 'normal']
neoplasia_data = data[data.Diagnosis == 'early neoplasia']
cancer_data = data[data.Diagnosis == 'cancer']

## Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

   1. для групп normal (control) и early neoplasia (treatment)
   2. для групп early neoplasia (control) и cancer (treatment)

In [4]:
genes = filter(lambda x: x not in ['Patient_id', 'Diagnosis'], data.columns.tolist())
control_vs_neoplasia = {}
neoplasia_vs_cancer = {}

In [5]:
for gene in genes:
    control_vs_neoplasia[gene] = (scipy.stats.ttest_ind(control_data[gene], 
                                                       neoplasia_data[gene], 
                                                       equal_var = False).pvalue)
    neoplasia_vs_cancer[gene] = (scipy.stats.ttest_ind(cancer_data[gene], 
                                                      neoplasia_data[gene], 
                                                      equal_var = False).pvalue)

In [8]:
num_diffs = sum(value < 0.05 for value in control_vs_neoplasia.values()) + sum(value < 0.05 for value in neoplasia_vs_cancer.values())  
num_diffs

<ipython-input-8-3ee907e18392>:1: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  num_diffs = sum(value < 0.05 for value in control_vs_neoplasia.values()) + sum(value < 0.05 for value in neoplasia_vs_cancer.values())


5065

In [27]:
control_vs_neoplasia_df = pd.DataFrame.from_dict(control_vs_neoplasia, orient = 'index')
control_vs_neoplasia_df.columns = ['pvalue']

neoplasia_vs_cancer_df = pd.DataFrame.from_dict(neoplasia_vs_cancer, orient = 'index')
neoplasia_vs_cancer_df.columns = ['pvalue']

control_vs_neoplasia_df[control_vs_neoplasia_df.pvalue < 0.05].shape

(1575, 1)

In [28]:
neoplasia_vs_cancer_df[neoplasia_vs_cancer_df.pvalue < 0.05].shape

(3490, 1)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

## Часть 2: поправка методом Холма

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

In [9]:
import statsmodels.stats.multitest as smm

In [18]:
reject, p_corrected, a1, a2 = multipletests(list(control_vs_neoplasia.values()), 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [20]:
p_corrected

array([1.        , 0.50017368, 1.        , ..., 1.        , 1.        ,
       1.        ])

In [32]:

reject, p_corrected, _, _ = multipletests(list(neoplasia_vs_cancer.values()), 
                                            alpha = 0.025, 
                                            method = 'holm') 

In [33]:
neoplasia_vs_cancer_df['p_corrected'] = p_corrected
neoplasia_vs_cancer_df['reject'] = rejec

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

In [35]:
def fold_change(c,t):
    if t > c:
        return t/c
    else:
        return -c/t

In [44]:
control_vs_neoplasia_df['fold_change'] = map(lambda x, y:fold_change(x,y),
                                             control_data.mean(),
                                             neoplasia_data.mean())
neoplasia_vs_cancer_df['fold_change'] = map(lambda x, y:fold_change(x,y),
                                            neoplasia_data.mean(),
                                            cancer_data.mean())

In [47]:
control_vs_neoplasia_df[control_vs_neoplasia_df.fold_change > 1.5].shape

AttributeError: 'Series' object has no attribute 'to_float'

In [42]:
neoplasia_vs_cancer_df[neoplasia_vs_cancer_df.fold_change > 1.5].shape

(451, 4)

## Часть 3: поправка методом Бенджамини-Хохберга